### Tool Calling With Agentic RAG Systems

In [2]:
import dotenv
%load_ext dotenv
%dotenv

In [3]:
import nest_asyncio
nest_asyncio.apply()

#### Sample Functions For Tools

In [3]:
def add(x: int, y: int) -> int:
    """Add two numbers together"""
    return x + y

def subtract(x: int, y: int) -> int:
    """Subtract two numbers"""
    return x - y

def multiply(x: int, y: int) -> int:
    """Multiply two numbers"""
    return x * y

def get_user_info(username: str) -> dict:
    """Get user information"""
    datanbase = {
        "john": {
            "name": "John Doe",
            "age": 25,
            "email": "johndoe@example.com"
        },
        "jane": {
            "name": "Jane Doe",
            "age": 20,
            "email": "janedoe@example.com"
        }
    }
    
    return f"Username: {username}, Info: {datanbase.get(username.lower(), 'User not found')}"

Crop Yeild Function

In [10]:
import datetime
def cropYeild(date: str, y: int) -> int:
    """Calculate crop yeild for date:2023-11-17T00:00:00.000Z"""
    """ Steps of Action ####

    ## 1. Trigger Data download from sentinel Hub OGC API using the input shapefile
    ## 2. accordingly download FAPAR, LSWI,  from sentinel Hub
    ## 3. Calculate W from LSWI
    ## 4. Use 24 hour sum og GHI and then it'll be Wh/m2 and then convert in kwh/m2
    ## 5. Using these LUE values as meyion and , calculate NPP
    ## 6. Mention HArvest Index (0.8) and caluclate final yield
    ## 7. Do a multiplication factor (10) to convert tons per hectare
    ## 8. Save pixel wise 
    ## 9. Perform zonal statistics and calculate avg yield and this will be final field wise yield (ton/hectare) """
    
    print("cropyieldscript")

    import sentinelhub
    from sentinelhub import SHConfig


    import datetime
    import numpy as np
    from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest
    import pyproj
    from IPython import get_ipython
    from sentinelhub import CustomUrlParam
    import warnings
    import geopandas as gpd
    import rasterio
    import rasterio.mask
    from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest, WebFeatureService, CustomUrlParam
    from rasterio import plot
    from PIL import Image
    import requests
    import json
    import pyproj
    import geopandas as gpd
    from rasterstats import zonal_stats
    from datetime import datetime, date
    warnings.filterwarnings("ignore")
    # FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
    #   lon, lat = pyproj.transform(utm_proj, latlong_proj, easting, northing)


    # get_ipython().system('sentinelhub.config --instance_id 924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d')
    # get_ipython().system('sentinelhub.config --show')


    date_path = r'date_data.json'
    with open(date_path, 'r') as f:
        data = json.load(f)

    stored_date = data['date']


    # Convert the stored_date string to datetime object
    stored_datetime = datetime.fromisoformat(stored_date.replace('Z', '+00:00'))

    # print(type(stored_datetime))
    # Format the stored_datetime to match the required time format
    formatted_time = stored_datetime.date().isoformat()
    print(formatted_time)
    INSTANCE_ID = '924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d'  # In case you put instance ID into configuration file you can leave this unchanged

    if INSTANCE_ID:
        config = SHConfig()
        config.instance_id = INSTANCE_ID
    else:
        config = None

    file_path = r'geojson_data.json'

    gdf = gpd.read_file(file_path)
    print(gdf)
    gdf.to_file('data/output.shp')
    tmp = gpd.read_file('data/output.shp')

    # print(tmp.head())

    with open(file_path, 'r') as f:
            geojson_data = json.load(f)

    # print((geojson_data))

    def read_geojson_data():
        with open(file_path, 'r') as f:
            geojson_data = json.load(f)
        return geojson_data

    def coordinates_calc():
        geojson_data = read_geojson_data()
        coordinates = geojson_data['geometry']['coordinates'][0]
        coordinates = coordinates[:-1]
        for i in range(len(coordinates)):
            coordinates[i][0], coordinates[i][1] = coordinates[i][1], coordinates[i][0]

        #print((coordinates))

        x_min, y_min = x_max, y_max = coordinates[0]

        # Find the maximum and minimum values in x and y coordinates
        for coordinate in coordinates:
            x, y = coordinate
            if x < x_min:
                x_min = x
            elif x > x_max:
                x_max = x
            if y < y_min:
                y_min = y
            elif y > y_max:
                y_max = y

        # Define the bounding box
        min_max = [x_min, y_min, x_max, y_max]
        print(min_max)
        src_crs = pyproj.CRS('EPSG:4326')  # WGS 84
        dst_crs = pyproj.CRS('EPSG:32644')  # UTM 44N

        # Define the coordinate transformation
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

        # Transform the coordinates to the UTM projection and units
        x_min, y_min = transformer.transform(min_max[0], min_max[1])
        x_max, y_max = transformer.transform(min_max[2], min_max[3])

        # Define the bounding box in the UTM projection and units
        bbbox = [x_min, y_min, x_max, y_max]
        # print(coordinates)
        # print((bbbox))
        inversed_geojson_data = geojson_data.copy()

    # Reverse the coordinate pairs using list comprehension
        inversed_coordinates = [
            [coord[1], coord[0]] for coord in geojson_data["geometry"]["coordinates"][0]
        ]
        # Replace the coordinates in the copied inversed_geojson_data
        inversed_geojson_data["geometry"]["coordinates"] = [inversed_coordinates]
        return bbbox
    # Use the GeoJSON data as needed



    # get_ipython().system('sentinelhub.config --show')
    # get_ipython().run_line_magic('reload_ext', 'autoreload')
    # get_ipython().run_line_magic('autoreload', '2')
    # get_ipython().run_line_magic('matplotlib', 'inline')


    def plot_image(image, factor=1):
        """
        Utility function for plotting RGB images.
        """
        fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

        if np.issubdtype(image.dtype, np.floating):
            plt.imshow(np.minimum(image * factor, 1))
        else:
            plt.imshow(image)


    def dimensions():
        extent = coordinates_calc()
        print(extent)
        bbox = BBox(bbox=extent, crs=CRS.UTM_44N)
        centroid_x = (extent[0] + extent[2]) / 2
        centroid_y = (extent[1] + extent[3]) / 2
        centroid = [centroid_x, centroid_y]
        FAPAR = fapar_layer_call(bbox)
        transform = rasterio.transform.from_bounds(extent[0], extent[1], extent[2], extent[3], FAPAR.shape[1], FAPAR.shape[0])
        return bbox, centroid, transform


    #list (CustomUrlParam)


    #SENTINELHUB API WCS REQUEST FOR FAPAR LAYER
    def fapar_layer_call(bbox):
        wcs_true_color_request = WcsRequest(
            data_collection = DataCollection.SENTINEL2_L2A,
            data_folder = 'data/',
            layer = 'FAPAR_1',
            bbox = bbox,
            time = formatted_time,
            resx = '10m',
            resy = '10m',
            image_format = MimeType.TIFF,
            custom_url_params = {
                CustomUrlParam.SHOWLOGO: False
            },
            config = config
        )
        wcs_true_color_img = wcs_true_color_request.get_data()
        FAPAR = wcs_true_color_img[0]
        return FAPAR


    #SENTINELHUB API WCS REQUEST FOR LSWI LAYER
    def LSWI_layer_call(bbox):
        wcs_true_color_request_LSWI = WcsRequest(
            data_collection = DataCollection.SENTINEL2_L2A,
            data_folder = 'data/',
            layer = 'LSWI',
            bbox = bbox,
            time =formatted_time,
            resx = '10m',
            resy = '10m',
            image_format = MimeType.TIFF,
            custom_url_params = {
                CustomUrlParam.SHOWLOGO: False
            },
            config = config
        )
        wcs_true_color_img = wcs_true_color_request_LSWI.get_data()
        LSWI = wcs_true_color_img[0]
        return LSWI


    # w calculations
    def w_calc():
        bbox, centroid, transform = dimensions()
        LSWI = LSWI_layer_call(bbox)
        LSWI_max = np.amax(LSWI)
        w = (1 - LSWI) / (1 + LSWI_max)
        return w, transform


    # w plotting and saving tiff file
    w, transform = w_calc()
    # print(np.amax(w))
    # print(np.amin(w))
    # plt.imshow(w, cmap = 'RdYlGn')
    # plt.show()
    # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/W.tiff', 'w', driver = 'GTiff', width = w.shape[1], height = w.shape[0], count = 1, dtype = w.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(w, 1)


    # LUE CALCULATIONS
    def lue_calc():
        e0 = 3.22
        w, transform = w_calc()
        LUE = e0 * w
        return LUE, transform


    # LUE PLOTTING AND SAVING TIFF FILE
    LUE, transform = lue_calc()
    # print(np.amax(LUE))
    # print(np.amin(LUE))
    # plt.imshow(LUE, cmap = 'RdYlGn')
    # plt.show()
    # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/LUE.tiff', 'w', driver = 'GTiff', width = LUE.shape[1], height = LUE.shape[0], count = 1, dtype = LUE.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(LUE, 1)


    # UTM TO LAT-LONG CONVERSION
    def utm_to_latlong(easting, northing, zone_number, zone_letter):
        utm_proj = pyproj.Proj(proj="utm", zone=zone_number, ellps="WGS84", south=("N" > zone_letter))
        latlong_proj = pyproj.Proj(proj="latlong", ellps="WGS84")
        lon, lat = pyproj.transform(utm_proj, latlong_proj, easting, northing)
        return lat, lon


    #GLOBAL HORIZONTAL IRRADIANCE CALCULATIONS
    def ghi_calc():
        bbox, centroid, transform = dimensions()
        date = formatted_time
        latitude, longitude = utm_to_latlong(centroid[0], centroid[1], 44, "R")
        lat = latitude
        lon = longitude
        API_KEY = 'eb8cf696ffe68ba33f4b7c3b25e45d5c' #OpenWeatherMap @ARMS4AI
        #-------Call this API Once-------
        response = requests.get(f"https://api.openweathermap.org/energy/1.0/solar/data?lat={lat}&lon={lon}&date={date}&appid={API_KEY}")
        data = response.json()

        with open('data.json', 'w') as f:
            json.dump(data, f)

        f = open('data.json')
        j= json.load(f)
        GHI=0
        for i in j["irradiance"]["daily"]:
            GHI = i['clear_sky']['ghi']
        GHI = GHI/1000
        GHI = GHI * 3.6
        print(GHI)
        return GHI


    # NPP CALCULATIONS
    def npp_calc():
        bbox, centroid, transform = dimensions()
        FAPAR = fapar_layer_call(bbox)
        GHI = ghi_calc()
        LUE, transform = lue_calc()
        NPP = FAPAR * GHI * 0.5 * LUE
        nodata = 0
        NPP[NPP < 0] = nodata
        return NPP


    # NPP PLOTTING AND SAVING TIFF FILE
    NPP = npp_calc()
    # plt.imshow(NPP, cmap = 'RdYlGn')
    # plt.show()
    # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/NPP.tiff', 'w', driver = 'GTiff', width = NPP.shape[1], height = NPP.shape[0], count = 1, dtype = NPP.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(NPP, 1)


    # CROP YIELD CALCIULATIONS
    def cy_calc():
        NPP = npp_calc()
        cy = NPP * 0.8 *10
        return cy



    # CROP YIELD PLOTTING AND SAVING TIFF FILE
    cy = cy_calc()
    # plt.imshow(cy, cmap = 'RdYlGn')
    # plt.show()
    with rasterio.open('data/cy.tiff', 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:32644', transform = transform) as dst:
        dst.write(cy, 1)
    # with rasterio.open(r'C:\Users\ANUBHAV\OneDrive\Desktop\Web App\reactAppArms4AI\reactapp\public\cyx.tiff', 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(cy, 1)

    #CLIPPING RASTER 
    shp = gpd.read_file("data/output.shp")         ##add path to shapefile
    shp = shp.to_crs(epsg = 32644)
    with rasterio.open("data/cy.tiff") as src:    ##add path to CY.tiff file
        out_image, out_transform = rasterio.mask.mask(src, shp.geometry,crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
        
    with rasterio.open("data/cy.tiff", "w", **out_meta) as dest:
        dest.write(out_image)

    # ZONAL STATISTICS
    def zonal_stats_calc():
        cy = zonal_stats(shp,
                "data/cy.tiff",
                band=1,
                            nodata=np.nan,
                            stats=['mean'], geojson_out=True)
        return cy[0]['properties']['mean']

    cy_mean = zonal_stats_calc() #send
    print(cy_mean)
    cy_mean = round(cy_mean,2)

    with open('cy_mean.json', 'w') as outfile:
        json.dump(cy_mean, outfile)
    return cy_mean


#### Creating Tools From Python Functions

In [11]:
from llama_index.core.tools import FunctionTool

cropyeild_tools = FunctionTool.from_defaults(fn=cropYeild)

# addition_tools = FunctionTool.from_defaults(fn=add)
# substitution_tools = FunctionTool.from_defaults(fn=subtract)
# multiplication_tools = FunctionTool.from_defaults(fn=multiply)
# get_user_info_tools = FunctionTool.from_defaults(fn=get_user_info)

# tools = [addition_tools, substitution_tools, multiplication_tools, get_user_info_tools]
tools = [cropyeild_tools]

#### Testing Out The Tool Calling

In [15]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

response = llm.predict_and_call(
    tools,
    "Calculate Crop Yeild for date 2023-11-17T00:00:00.000Z and crop: Sugarcane",
    verbose=True
)

print(str(response))

=== Calling Function ===
Calling function: cropYeild with args: {"date": "2023-11-17T00:00:00.000Z", "y": 2023}
cropyieldscript
2023-11-17
                                            geometry
0  POLYGON ((80.20980 27.38839, 80.20980 27.40645...
[27.388392, 80.209796, 27.406447, 80.238112]
[421869.46740560315, 3029702.2767849867, 424681.51621121203, 3031684.804159714]
[27.388392, 80.209796, 27.406447, 80.238112]
[421869.46740560315, 3029702.2767849867, 424681.51621121203, 3031684.804159714]
[27.388392, 80.209796, 27.406447, 80.238112]
[421869.46740560315, 3029702.2767849867, 424681.51621121203, 3031684.804159714]
[27.388392, 80.209796, 27.406447, 80.238112]
[421869.46740560315, 3029702.2767849867, 424681.51621121203, 3031684.804159714]
16.691652
[27.388392, 80.209796, 27.406447, 80.238112]
[421869.46740560315, 3029702.2767849867, 424681.51621121203, 3031684.804159714]
[27.388392, 80.209796, 27.406447, 80.238112]
[421869.46740560315, 3029702.2767849867, 424681.51621121203, 3031684.804159

In [12]:
from openai import OpenAI

client = OpenAI()

date = "2023-11-17"
crop = "Sugarcane"

prompt = f"Calculate Crop Yield for date {date} and crop: {crop}. Consider the following factors: weather patterns, soil conditions, historical data, etc."


stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    tools,
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

SyntaxError: positional argument follows keyword argument (4130694850.py, line 16)

In [13]:
from openai import OpenAI
client = OpenAI()
 
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-3.5-turbo",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
  ]
)

In [14]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the weather in San Francisco today and the likelihood it'll rain?",
)

In [15]:
from typing_extensions import override
from openai import AssistantEventHandler
 
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
      # Retrieve events that are denoted with 'requires_action'
      # since these will have our tool_calls
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
        if tool.function.name == "get_current_temperature":
          tool_outputs.append({"tool_call_id": tool.id, "output": "57"})
        elif tool.function.name == "get_rain_probability":
          tool_outputs.append({"tool_call_id": tool.id, "output": "0.06"})
        
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        for text in stream.text_deltas:
          print(text, end="", flush=True)
        print()
 
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()

The current temperature in San Francisco is 57°C, and there is a 6% likelihood of rain today.



In [5]:
from openai import OpenAI
# Set up the OpenAI API key
#openai.api_key = "your-openai-api-key"

# Define the input parameters
date = "2023-11-17"
crop = "Sugarcane"

# Prepare the prompt for the model
prompt = f"Calculate Crop Yield for date {date} and crop: {crop}. Consider the following factors: weather patterns, soil conditions, historical data, etc."
client = OpenAI()
# Make the API call using the new interface
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": prompt},
        stream=True
    ]
)
for chunk in response:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3546782166.py, line 18)

In [12]:
response = llm.predict_and_call(
    tools,
    "Tell me how old is John.",
    verbose=True
)

print(str(response))

=== Calling Function ===
Calling function: get_user_info with args: {"username": "John"}
=== Function Output ===
Username: John, Info: {'name': 'John Doe', 'age': 25, 'email': 'johndoe@example.com'}
Username: John, Info: {'name': 'John Doe', 'age': 25, 'email': 'johndoe@example.com'}


#### Vector Search With Metadata

In [15]:
from llama_index.core import SimpleDirectoryReader


# read in lora paper
documents = SimpleDirectoryReader(input_files=["./datasets/lora_paper.pdf"]).load_data()

In [16]:
from llama_index.core.node_parser import SentenceSplitter


splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents=documents)

In [17]:
len(nodes)

38

In [19]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embedding = OpenAIEmbedding(model="text-embedding-ada-002")

In [21]:
from llama_index.core import VectorStoreIndex


vector_index = VectorStoreIndex(nodes=nodes)

In [24]:
from llama_index.core.vector_stores import MetadataFilters


query_engine = vector_index.as_query_engine(
    similarity_top_k=3,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)


response = query_engine.query("Tell me abou the problem statement as explained in page 2")
print(str(response))   # print the response

The problem statement focuses on language modeling, particularly in the context of adapting a pre-trained autoregressive language model to downstream conditional text generation tasks. The goal is to maximize conditional probabilities given a task-specific prompt. The pre-trained model, such as GPT based on the Transformer architecture, is adapted for tasks like summarization, machine reading comprehension, and natural language to SQL. Each task involves a training dataset of context-target pairs, where the context and target are sequences of tokens. For instance, in NL2SQL, the context is a natural language query and the target is the corresponding SQL command; in summarization, the context is an article's content and the target is its summary.


In [29]:
for n in response.source_nodes:
    print(n.metadata)
    print("+++++++++++++")
    print(n.get_text())
    print("+++++++++++++")

{'page_label': '2', 'file_name': 'lora_paper.pdf', 'file_path': 'datasets/lora_paper.pdf', 'file_type': 'application/pdf', 'file_size': 1609513, 'creation_date': '2024-05-12', 'last_modified_date': '2024-05-12'}
+++++++++++++
often introduce inference latency (Houlsby et al., 2019; Rebufﬁ et al., 2017) by extending model
depth or reduce the model’s usable sequence length (Li & Liang, 2021; Lester et al., 2021; Ham-
bardzumyan et al., 2020; Liu et al., 2021) (Section 3). More importantly, these method often fail to
match the ﬁne-tuning baselines, posing a trade-off between efﬁciency and model quality.
We take inspiration from Li et al. (2018a); Aghajanyan et al. (2020) which show that the learned
over-parametrized models in fact reside on a low intrinsic dimension. We hypothesize that the
change in weights during model adaptation also has a low “intrinsic rank”, leading to our proposed
Low-RankAdaptation (LoRA) approach. LoRA allows us to train some dense layers in a neural
network indi

In [30]:
from typing import List
from llama_index.core.vector_stores import FilterCondition

def vector_search_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Conduct a vector search across an index using the following parameters:

    query (str): This is the text string you want to embed and search for within the index.
    page_numbers (List[str]): This parameter allows you to limit the search to 
    specific pages. If left empty, the search will encompass all pages in the index. 
    If page numbers are specified, the search will be filtered to only include those pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response

In [31]:
vector_query_tool = FunctionTool.from_defaults(
    fn=vector_search_query, 
    name="Vector_search_query_tool"
)

In [32]:
response = llm.predict_and_call(
    [vector_query_tool],
    "Tell me about the problem statement as explained in page 2",
    verbose=True
)

=== Calling Function ===
Calling function: Vector_search_query_tool with args: {"query": "problem statement", "page_numbers": ["2"]}
=== Function Output ===
The problem statement focuses on language modeling, particularly on maximizing conditional probabilities given a task-specific prompt. It discusses adapting a pre-trained autoregressive language model to downstream conditional text generation tasks like summarization, machine reading comprehension, and natural language to SQL. Each task is defined by a dataset of context-target pairs, where the context is a sequence of tokens and the target is the corresponding output, such as a SQL command for a natural language query in NL2SQL or a summary for an article in summarization.


In [33]:
for n in response.source_nodes:
    print(n.metadata)
    print("+++++++++++++")
    print(n.get_text())
    print("+++++++++++++")

{'page_label': '2', 'file_name': 'lora_paper.pdf', 'file_path': 'datasets/lora_paper.pdf', 'file_type': 'application/pdf', 'file_size': 1609513, 'creation_date': '2024-05-12', 'last_modified_date': '2024-05-12'}
+++++++++++++
often introduce inference latency (Houlsby et al., 2019; Rebufﬁ et al., 2017) by extending model
depth or reduce the model’s usable sequence length (Li & Liang, 2021; Lester et al., 2021; Ham-
bardzumyan et al., 2020; Liu et al., 2021) (Section 3). More importantly, these method often fail to
match the ﬁne-tuning baselines, posing a trade-off between efﬁciency and model quality.
We take inspiration from Li et al. (2018a); Aghajanyan et al. (2020) which show that the learned
over-parametrized models in fact reside on a low intrinsic dimension. We hypothesize that the
change in weights during model adaptation also has a low “intrinsic rank”, leading to our proposed
Low-RankAdaptation (LoRA) approach. LoRA allows us to train some dense layers in a neural
network indi

In [37]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool


summary_index = SummaryIndex(nodes=nodes)

summary_query_eginetool = summary_index.as_query_engine(
    use_async=True,
    response_mode="tree_summarize"
)

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_eginetool,
    name="Summary_tool",
    description="Useful for summarization of the lora paper."
)

In [40]:
response = llm.predict_and_call(
    [summary_tool, vector_query_tool],
    "Tell me about the lora paper in a summary format.",
    verbose=True
)

=== Calling Function ===
Calling function: Summary_tool with args: {"input": "lora paper"}
=== Function Output ===
The LoRA paper introduces a method called Low-Rank Adaptation (LoRA) for efficiently adapting large language models to specific tasks. This approach involves freezing the pre-trained model weights and incorporating trainable rank decomposition matrices into each layer of the Transformer architecture. By doing so, LoRA significantly reduces the number of trainable parameters for downstream tasks, maintaining high model quality without adding inference latency. The paper demonstrates the effectiveness of LoRA on various language models like RoBERTa, DeBERTa, GPT-2, and GPT-3, showing its capability to outperform or match traditional fine-tuning methods while decreasing the number of trainable parameters and memory requirements.


In [41]:
print(str(response))

The LoRA paper introduces a method called Low-Rank Adaptation (LoRA) for efficiently adapting large language models to specific tasks. This approach involves freezing the pre-trained model weights and incorporating trainable rank decomposition matrices into each layer of the Transformer architecture. By doing so, LoRA significantly reduces the number of trainable parameters for downstream tasks, maintaining high model quality without adding inference latency. The paper demonstrates the effectiveness of LoRA on various language models like RoBERTa, DeBERTa, GPT-2, and GPT-3, showing its capability to outperform or match traditional fine-tuning methods while decreasing the number of trainable parameters and memory requirements.


In [42]:
for n in response.source_nodes:
    print(n.metadata)
    print("+++++++++++++")
    print(n.get_text())
    print("+++++++++++++")

{'page_label': '1', 'file_name': 'lora_paper.pdf', 'file_path': 'datasets/lora_paper.pdf', 'file_type': 'application/pdf', 'file_size': 1609513, 'creation_date': '2024-05-12', 'last_modified_date': '2024-05-12'}
+++++++++++++
LORA: L OW-RANK ADAPTATION OF LARGE LAN-
GUAGE MODELS
Edward Hu∗Yelong Shen∗Phillip Wallis Zeyuan Allen-Zhu
Yuanzhi Li Shean Wang Lu Wang Weizhu Chen
Microsoft Corporation
{edwardhu, yeshe, phwallis, zeyuana,
yuanzhil, swang, luw, wzchen }@microsoft.com
yuanzhil@andrew.cmu.edu
(Version 2)
ABSTRACT
An important paradigm of natural language processing consists of large-scale pre-
training on general domain data and adaptation to particular tasks or domains. As
we pre-train larger models, full ﬁne-tuning, which retrains all model parameters,
becomes less feasible. Using GPT-3 175B as an example – deploying indepen-
dent instances of ﬁne-tuned models, each with 175B parameters, is prohibitively
expensive. We propose Low-RankAdaptation, or LoRA, which freezes the pre-


In [ ]:
import datetime
def cropYeild():
    """Calculate crop yeild for date:2023-11-17T00:00:00.000Z"""
    """ Steps of Action ####

    ## 1. Trigger Data download from sentinel Hub OGC API using the input shapefile
    ## 2. accordingly download FAPAR, LSWI,  from sentinel Hub
    ## 3. Calculate W from LSWI
    ## 4. Use 24 hour sum og GHI and then it'll be Wh/m2 and then convert in kwh/m2
    ## 5. Using these LUE values as meyion and , calculate NPP
    ## 6. Mention HArvest Index (0.8) and caluclate final yield
    ## 7. Do a multiplication factor (10) to convert tons per hectare
    ## 8. Save pixel wise 
    ## 9. Perform zonal statistics and calculate avg yield and this will be final field wise yield (ton/hectare) """
    
    print("cropyieldscript")

    import sentinelhub
    from sentinelhub import SHConfig


    import datetime
    import numpy as np
    from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest
    import pyproj
    from IPython import get_ipython
    from sentinelhub import CustomUrlParam
    import warnings
    import geopandas as gpd
    import rasterio
    import rasterio.mask
    from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, WmsRequest, WebFeatureService, CustomUrlParam
    from rasterio import plot
    from PIL import Image
    import requests
    import json
    import pyproj
    import geopandas as gpd
    from rasterstats import zonal_stats
    from datetime import datetime, date
    warnings.filterwarnings("ignore")
    # FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
    #   lon, lat = pyproj.transform(utm_proj, latlong_proj, easting, northing)


    # get_ipython().system('sentinelhub.config --instance_id 924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d')
    # get_ipython().system('sentinelhub.config --show')


    date_path = r'date_data.json'
    with open(date_path, 'r') as f:
        data = json.load(f)

    stored_date = data['date']


    # Convert the stored_date string to datetime object
    stored_datetime = datetime.fromisoformat(stored_date.replace('Z', '+00:00'))

    # print(type(stored_datetime))
    # Format the stored_datetime to match the required time format
    formatted_time = stored_datetime.date().isoformat()
    print(formatted_time)
    INSTANCE_ID = '924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d'  # In case you put instance ID into configuration file you can leave this unchanged

    if INSTANCE_ID:
        config = SHConfig()
        config.instance_id = INSTANCE_ID
    else:
        config = None

    file_path = r'geojson_data.json'

    gdf = gpd.read_file(file_path)
    print(gdf)
    gdf.to_file('data/output.shp')
    tmp = gpd.read_file('data/output.shp')

    # print(tmp.head())

    with open(file_path, 'r') as f:
            geojson_data = json.load(f)

    # print((geojson_data))

    def read_geojson_data():
        with open(file_path, 'r') as f:
            geojson_data = json.load(f)
        return geojson_data

    def coordinates_calc():
        geojson_data = read_geojson_data()
        coordinates = geojson_data['geometry']['coordinates'][0]
        coordinates = coordinates[:-1]
        for i in range(len(coordinates)):
            coordinates[i][0], coordinates[i][1] = coordinates[i][1], coordinates[i][0]

        #print((coordinates))

        x_min, y_min = x_max, y_max = coordinates[0]

        # Find the maximum and minimum values in x and y coordinates
        for coordinate in coordinates:
            x, y = coordinate
            if x < x_min:
                x_min = x
            elif x > x_max:
                x_max = x
            if y < y_min:
                y_min = y
            elif y > y_max:
                y_max = y

        # Define the bounding box
        min_max = [x_min, y_min, x_max, y_max]
        print(min_max)
        src_crs = pyproj.CRS('EPSG:4326')  # WGS 84
        dst_crs = pyproj.CRS('EPSG:32644')  # UTM 44N

        # Define the coordinate transformation
        transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

        # Transform the coordinates to the UTM projection and units
        x_min, y_min = transformer.transform(min_max[0], min_max[1])
        x_max, y_max = transformer.transform(min_max[2], min_max[3])

        # Define the bounding box in the UTM projection and units
        bbbox = [x_min, y_min, x_max, y_max]
        # print(coordinates)
        # print((bbbox))
        inversed_geojson_data = geojson_data.copy()

    # Reverse the coordinate pairs using list comprehension
        inversed_coordinates = [
            [coord[1], coord[0]] for coord in geojson_data["geometry"]["coordinates"][0]
        ]
        # Replace the coordinates in the copied inversed_geojson_data
        inversed_geojson_data["geometry"]["coordinates"] = [inversed_coordinates]
        return bbbox
    # Use the GeoJSON data as needed



    # get_ipython().system('sentinelhub.config --show')
    # get_ipython().run_line_magic('reload_ext', 'autoreload')
    # get_ipython().run_line_magic('autoreload', '2')
    # get_ipython().run_line_magic('matplotlib', 'inline')


    def plot_image(image, factor=1):
        """
        Utility function for plotting RGB images.
        """
        fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

        if np.issubdtype(image.dtype, np.floating):
            plt.imshow(np.minimum(image * factor, 1))
        else:
            plt.imshow(image)


    def dimensions():
        extent = coordinates_calc()
        print(extent)
        bbox = BBox(bbox=extent, crs=CRS.UTM_44N)
        centroid_x = (extent[0] + extent[2]) / 2
        centroid_y = (extent[1] + extent[3]) / 2
        centroid = [centroid_x, centroid_y]
        FAPAR = fapar_layer_call(bbox)
        transform = rasterio.transform.from_bounds(extent[0], extent[1], extent[2], extent[3], FAPAR.shape[1], FAPAR.shape[0])
        return bbox, centroid, transform


    #list (CustomUrlParam)


    #SENTINELHUB API WCS REQUEST FOR FAPAR LAYER
    def fapar_layer_call(bbox):
        wcs_true_color_request = WcsRequest(
            data_collection = DataCollection.SENTINEL2_L2A,
            data_folder = 'data/',
            layer = 'FAPAR_1',
            bbox = bbox,
            time = formatted_time,
            resx = '10m',
            resy = '10m',
            image_format = MimeType.TIFF,
            custom_url_params = {
                CustomUrlParam.SHOWLOGO: False
            },
            config = config
        )
        wcs_true_color_img = wcs_true_color_request.get_data()
        FAPAR = wcs_true_color_img[0]
        return FAPAR


    #SENTINELHUB API WCS REQUEST FOR LSWI LAYER
    def LSWI_layer_call(bbox):
        wcs_true_color_request_LSWI = WcsRequest(
            data_collection = DataCollection.SENTINEL2_L2A,
            data_folder = 'data/',
            layer = 'LSWI',
            bbox = bbox,
            time =formatted_time,
            resx = '10m',
            resy = '10m',
            image_format = MimeType.TIFF,
            custom_url_params = {
                CustomUrlParam.SHOWLOGO: False
            },
            config = config
        )
        wcs_true_color_img = wcs_true_color_request_LSWI.get_data()
        LSWI = wcs_true_color_img[0]
        return LSWI


    # w calculations
    def w_calc():
        bbox, centroid, transform = dimensions()
        LSWI = LSWI_layer_call(bbox)
        LSWI_max = np.amax(LSWI)
        w = (1 - LSWI) / (1 + LSWI_max)
        return w, transform


    # w plotting and saving tiff file
    w, transform = w_calc()
    # print(np.amax(w))
    # print(np.amin(w))
    # plt.imshow(w, cmap = 'RdYlGn')
    # plt.show()
    # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/W.tiff', 'w', driver = 'GTiff', width = w.shape[1], height = w.shape[0], count = 1, dtype = w.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(w, 1)


    # LUE CALCULATIONS
    def lue_calc():
        e0 = 3.22
        w, transform = w_calc()
        LUE = e0 * w
        return LUE, transform


    # LUE PLOTTING AND SAVING TIFF FILE
    LUE, transform = lue_calc()
    # print(np.amax(LUE))
    # print(np.amin(LUE))
    # plt.imshow(LUE, cmap = 'RdYlGn')
    # plt.show()
    # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/LUE.tiff', 'w', driver = 'GTiff', width = LUE.shape[1], height = LUE.shape[0], count = 1, dtype = LUE.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(LUE, 1)


    # UTM TO LAT-LONG CONVERSION
    def utm_to_latlong(easting, northing, zone_number, zone_letter):
        utm_proj = pyproj.Proj(proj="utm", zone=zone_number, ellps="WGS84", south=("N" > zone_letter))
        latlong_proj = pyproj.Proj(proj="latlong", ellps="WGS84")
        lon, lat = pyproj.transform(utm_proj, latlong_proj, easting, northing)
        return lat, lon


    #GLOBAL HORIZONTAL IRRADIANCE CALCULATIONS
    def ghi_calc():
        bbox, centroid, transform = dimensions()
        date = formatted_time
        latitude, longitude = utm_to_latlong(centroid[0], centroid[1], 44, "R")
        lat = latitude
        lon = longitude
        API_KEY = 'eb8cf696ffe68ba33f4b7c3b25e45d5c' #OpenWeatherMap @ARMS4AI
        #-------Call this API Once-------
        response = requests.get(f"https://api.openweathermap.org/energy/1.0/solar/data?lat={lat}&lon={lon}&date={date}&appid={API_KEY}")
        data = response.json()

        with open('data.json', 'w') as f:
            json.dump(data, f)

        f = open('data.json')
        j= json.load(f)
        GHI=0
        for i in j["irradiance"]["daily"]:
            GHI = i['clear_sky']['ghi']
        GHI = GHI/1000
        GHI = GHI * 3.6
        print(GHI)
        return GHI


    # NPP CALCULATIONS
    def npp_calc():
        bbox, centroid, transform = dimensions()
        FAPAR = fapar_layer_call(bbox)
        GHI = ghi_calc()
        LUE, transform = lue_calc()
        NPP = FAPAR * GHI * 0.5 * LUE
        nodata = 0
        NPP[NPP < 0] = nodata
        return NPP


    # NPP PLOTTING AND SAVING TIFF FILE
    NPP = npp_calc()
    # plt.imshow(NPP, cmap = 'RdYlGn')
    # plt.show()
    # with rasterio.open('/Users/sid/Desktop/Sugarcane_Sentinel-2_Products/Crop_Yield/NPP.tiff', 'w', driver = 'GTiff', width = NPP.shape[1], height = NPP.shape[0], count = 1, dtype = NPP.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(NPP, 1)


    # CROP YIELD CALCIULATIONS
    def cy_calc():
        NPP = npp_calc()
        cy = NPP * 0.8 *10
        return cy



    # CROP YIELD PLOTTING AND SAVING TIFF FILE
    cy = cy_calc()
    # plt.imshow(cy, cmap = 'RdYlGn')
    # plt.show()
    with rasterio.open('data/cy.tiff', 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:32644', transform = transform) as dst:
        dst.write(cy, 1)
    # with rasterio.open(r'C:\Users\ANUBHAV\OneDrive\Desktop\Web App\reactAppArms4AI\reactapp\public\cyx.tiff', 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:32644', transform = transform) as dst:
    #     dst.write(cy, 1)

    #CLIPPING RASTER 
    shp = gpd.read_file("data/output.shp")         ##add path to shapefile
    shp = shp.to_crs(epsg = 32644)
    with rasterio.open("data/cy.tiff") as src:    ##add path to CY.tiff file
        out_image, out_transform = rasterio.mask.mask(src, shp.geometry,crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
        
    with rasterio.open("data/cy.tiff", "w", **out_meta) as dest:
        dest.write(out_image)

    # ZONAL STATISTICS
    def zonal_stats_calc():
        cy = zonal_stats(shp,
                "data/cy.tiff",
                band=1,
                            nodata=np.nan,
                            stats=['mean'], geojson_out=True)
        return cy[0]['properties']['mean']

    cy_mean = zonal_stats_calc() #send
    print(cy_mean)
    cy_mean = round(cy_mean,2)

    with open('cy_mean.json', 'w') as outfile:
        json.dump(cy_mean, outfile)
    return cy_mean
